# A pytorch ML adapter demo

 1. [Pytorch](#pytorch) model example.
 2. Create [Adapter](#adapter).
 3. How to create a [Webscript](#webscript).
 4. How to create a [Plug](#plug).

#### Preamble
Demo of a relatively small pytorch model.
This notebook uses the [ml_adapter_torch](../../env/ml_adapter_torch) dependencies.
To start: 
```
bin/jupyter_notebook env/ml_adapter_torch ml_adapter/torch_autoencoder
```

In [1]:
# the sdk profile used to connect
PROFILE='_default_'
LOG_LEVEL='INFO'
MODEL_NAME='autoencoderV1'
MODEL_VERSION='1.0.0'

In [2]:
# setup INFO logging to see http requests made.
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=LOG_LEVEL,
    datefmt='%Y-%m-%d %H:%M:%S'
)

## 1. Pytorch example <a id="pytorch"></a>

A simple auto-encoder pytorch model.

In [3]:
from autoencoder import AutoEncoder

In [4]:
# we saved our model class in a `autoencoder.py` file
from IPython.display import Code, Markdown
display(Markdown(f'### `autoencoder.py` source'))
display(Code(filename='autoencoder.py'))
from autoencoder import AutoEncoder

### `autoencoder.py` source

import torch

class AutoEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        """
        Create a simple AutoEncoder
        """
        # Use an AutoEncoder and try to reconstruct both signals (the 20 samples back)
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(20, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 5),
        )

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(5, 10),
            torch.nn.ReLU(),
            torch.nn.Linear(10, 20),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
# some pretrained weights
weights_path = 'AutoEncoderWeights.pth'

In [6]:
import torch
model = AutoEncoder()
model.load_state_dict(torch.load(weights_path, weights_only=True))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=5, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=20, bias=True)
  )
)

In [7]:
x_data = torch.randn(20, dtype=torch.float32)
x_data

tensor([ 1.1458, -0.8573, -0.4810,  1.3070, -1.6287, -2.4500, -2.0896,  0.5397,
        -0.2585,  0.6075, -0.3521,  0.5024, -0.0205,  0.2033,  0.6291,  0.6521,
        -1.6579,  0.0916, -0.2855, -1.4349])

In [8]:
# a model inference:
preds = model(x_data)
preds

tensor([-0.3229, -0.3600, -0.3960, -0.3263, -0.3969, -0.4243, -0.4064, -0.3829,
        -0.4136, -0.2749, -0.3954, -0.3799, -0.3340, -0.3710, -0.3791, -0.3988,
        -0.3101, -0.4098, -0.4013, -0.3952], grad_fn=<ViewBackward0>)

## 2. The adapter <a id="adapter"></a>
The `V1TorchAdapter` from the `ml_adapter.torch` module wraps our model in a script that can be used as a waylay webscript or plug.

In [9]:
# choose a local archive location
ARCHIVE_LOC = 'autoencoder-pytorch'
# make sure its empty
!rm -fr autoencoder-pytorch

In [10]:
from ml_adapter.torch import V1TorchAdapter
# create an ML adapter to wrap our model
# by using a `weights.pt` postfix we are storing only the weights when serializing the model
MODEL_PATH='model-weights.pt'
adapter = V1TorchAdapter(model=model, model_path='model-weights.pt', location=ARCHIVE_LOC)

In [11]:
# the adapter exposes your model with a REST-compatible interface
result = await adapter.call({"instances": [ x_data.tolist() ]}) 
result

{'predictions': [[-0.3229316771030426,
   -0.36001601815223694,
   -0.3960452079772949,
   -0.32631662487983704,
   -0.3968595266342163,
   -0.4243429899215698,
   -0.40644288063049316,
   -0.3829203248023987,
   -0.41355741024017334,
   -0.27493229508399963,
   -0.3954361081123352,
   -0.3798944652080536,
   -0.3340391516685486,
   -0.3709794878959656,
   -0.37912052869796753,
   -0.3987690806388855,
   -0.3101225197315216,
   -0.40983518958091736,
   -0.40134039521217346,
   -0.3952208459377289]]}

In [12]:
# because we store only weights, the adapter archive needs to now about autoencode model class
# Its not recommended to store full serialized models, as these are more brittle with respect versions of python and torch
await adapter.add_script('autoencoder.py')

autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>

In [13]:
await adapter.save()
# have a look at ARCHIVE_LOC to see the stored assets
list(a.path for a in adapter.assets)

['model-weights.pt',
 'webscript.json',
 'requirements.txt',
 'main.py',
 'autoencoder.py']

## 3. Deploying as webscript <a id="webscript"></a>
Tell the adapter to configure itself as a webscript: this generates a number of _assets_ that will be uploaded and define the webscript behaviour
* a `webscript.json` _manifest_ file that defines the name, version, _runtime_, deploy settings, metadata ...
* a `requirements.txt` package dependencies file
* a `main.py` webscript script
* additional scripts we added above, like the `autoencoder.py` and the `model-weights.pt`

In [14]:
## configure any 'memory' or 'cpu' deploy settings 
deploy_overrides = {'limits' : { 'memory': '2G' }, 'requests' : { 'memory' : '1G' }}
## configure the webscript to use our model
adapter = adapter.as_webscript({
    'name': MODEL_NAME, 
    'description':'pytorch autoencoder for caats', 
    'deploy' : deploy_overrides                                                                                          
})

In [15]:
await adapter.save()
list(a.path for a in adapter.assets)

['model-weights.pt',
 'webscript.json',
 'requirements.txt',
 'main.py',
 'autoencoder.py']

In [16]:
# lets have a look at the generated python webscript:
display(Code(filename=f'{ARCHIVE_LOC}/main.py'))

# You could adapt this script to have specific error handling or handling of request/response

# ml_adapter.torch.adapter.V1TorchAdapter model adapter
import os
from starlette.requests import Request
from starlette.responses import JSONResponse
from starlette.exceptions import HTTPException
from ml_adapter.torch.adapter import V1TorchAdapter

MODEL_PATH = os.environ.get('MODEL_PATH', 'model-weights.pt')
MODEL_CLASS = os.environ.get('MODEL_CLASS', 'autoencoder.AutoEncoder')

# Initialize the model adapter.
# Provide a `model` argument if you want to create/load the model yourself.
adapter = V1TorchAdapter(
    model_path=MODEL_PATH, model_class=MODEL_CLASS
)

# Webscript handler
async def execute(request: Request):
    if request.method == 'GET':
        return JSONResponse(adapter.openapi)
    if request.method != 'POST':
        raise HTTPException(
            status_code=405,
            detail='This webscript only accepts `POST` calls.',
        )
    # use request body as input
    request_json = await request.json()
    # call the model adapter using the V1
    response_json = await adapter.call(request_json)
    return JSONResponse(response_json)

In [17]:
# once stored, the adapter can be restored later with
adapter = await V1TorchAdapter(model_path='model-weights.pt', model_class=AutoEncoder, location=ARCHIVE_LOC).load()

2025-01-21 17:19:54 INFO     loading torch model weights from autoencoder-pytorch/model-weights.pt
2025-01-21 17:19:54 INFO     creating torch model with class AutoEncoder
2025-01-21 17:19:54 INFO     loading torch model weights from autoencoder-pytorch/model-weights.pt
2025-01-21 17:19:54 INFO     creating torch model with class AutoEncoder


In [18]:
list(adapter.assets)

[model-weights.pt <ml_adapter.torch.adapter.TorchModelWeightsAsset>,
 webscript.json <ml_adapter.base.assets.manifest.WebscriptManifestAsset>,
 requirements.txt <ml_adapter.base.assets.python.PythonRequirementsAsset>,
 main.py <ml_adapter.base.assets.python.PythonScriptAsset>,
 autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>]

### Uploading the webscript using the SDK
To upload these assets and create a webscript, we need to call the [create webcript](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Webscripts/operation/create_webscripts) REST api. 

The code belows uses the `ml_tool` plugin to handle this.
Alternatively you could call `await adapter.save_archive()`
which creates an `autoencoder-pytorch.tar.gz` archive that you can upload as webscript with `curl` or using the waylay console.

In [19]:
from waylay.sdk import WaylayClient

In [20]:
# check the SDK client
client = WaylayClient.from_profile('staging')

In [21]:
import asyncio
try:
    ref = await client.registry.webscripts.remove_versions(MODEL_NAME)
    await asyncio.sleep(5)
except Exception as e:
    print(f'nothing to delete? {e}')

2025-01-21 17:19:54 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/accounts/v1/tokens?grant_type=client_credentials "HTTP/1.1 200 OK"
2025-01-21 17:19:55 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/webscripts/autoencoderV1 "HTTP/1.1 202 Accepted"


In [22]:
ref = await client.ml_tool.create_webscript(adapter)
ref

2025-01-21 17:20:05 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/webscripts/?draft=false&comment=&async=true "HTTP/1.1 202 Accepted"


{'message': "Building and deploying webscript 'autoencoderV1@0.0.1'",
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$6mcNQQ8pV1oC-frgiwTZL&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/verify/740799ef-d515-4704-8718-903851c9899e$6mcNQQ8pV1oC-frgiwTZL'}},
 'entity': {'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2025-01-21T16:20:05.603Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2025-01-21T16:20:05.615Z',
  'status': 'pending',
  'runtime': {'deprecated': False,
   'upgradable': False,
   'name': 'web-python3',
   'version': '0.2.0'},
  'deprecated': False,
  'draft': False,
  'webscript': {'name': 'autoencoderV1',
   'version': '0.0.1',
   'runtime': 'web-python3',
   'metadata': {},
   'private': True,
   'allowHmac': True,
   'deploy': {'limits': {'memory': '2G'}, 'requests': {'memory': '1G'}},


In [23]:
# get the current status
# ref = await client.registry.webscripts.get(MODEL_NAME,'0.0.1', response_type=dict)

In [24]:
# when build/deployment fails, this performs a retry
# ref = await client.registry.webscripts.rebuild(MODEL_NAME,'0.0.1', query={'ignoreChecks':True}, response_type=dict)

In [25]:
# wait until the build, deploy and verify jobs for the webscript have finished
# NOTE: this might take quite a few minutes, as the dependencies for torch webscripts are quite big and the images not yet optimised
ref = await client.ml_tool.wait_until_ready(ref)

2025-01-21 17:20:05 INFO     Waiting for autoencoderV1@0.0.1 to be ready:
2025-01-21 17:20:05 INFO     listening on https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$6mcNQQ8pV1oC-frgiwTZL&children=true
2025-01-21 17:20:05 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$6mcNQQ8pV1oC-frgiwTZL&children=true "HTTP/1.1 200 OK"
2025-01-21 17:20:05 INFO     ack: Listening to events of jobs dependent on job 740799ef-d515-4704-8718-903851c9899e$6mcNQQ8pV1oC-frgiwTZL
2025-01-21 17:20:05 INFO     autoencoderV1@0.0.1 build: active
2025-01-21 17:20:06 INFO     autoencoderV1@0.0.1 build: completed
{'data': {'returnvalue': {'digest': 'e73d693ccbed523f67e935efa41ae61b6969b3df3969ba75075c129ef69fd4ee', 'log': [], 'status': 'success'}}, 'job': {'id': '740799ef-d515-4704-8718-903851c9899e$IGqkit2bCuxsobW9x2iKQ', 'type': 'build'}, 'timestamp': '2025-01-21T16:20:06.70

In [26]:
# test the webscript invocation
await client.ml_tool.test_webscript(ref, x_data.tolist())

2025-01-21 17:22:09 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/functions/v1/740799ef-d515-4704-8718-903851c9899e/autoencoderV1 "HTTP/1.1 200 OK"


[-0.3229316174983978,
 -0.36001601815223694,
 -0.39604517817497253,
 -0.32631659507751465,
 -0.3968595266342163,
 -0.4243429899215698,
 -0.4064428508281708,
 -0.3829203248023987,
 -0.41355741024017334,
 -0.27493223547935486,
 -0.3954361081123352,
 -0.3798944652080536,
 -0.3340390920639038,
 -0.3709794282913208,
 -0.37912046909332275,
 -0.3987690806388855,
 -0.31012246012687683,
 -0.40983518958091736,
 -0.4013403654098511,
 -0.3952208161354065]

In [27]:
# remove the webscript
await client.ml_tool.remove(ref)

2025-01-21 17:22:09 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/webscripts/autoencoderV1/versions/0.0.1?force=true "HTTP/1.1 202 Accepted"


{'message': "Removing webscript version 'autoencoderV1@0.0.1'",
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=undeploy&id=740799ef-d515-4704-8718-903851c9899e$wjw76aPILglewoYdbq4KK&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/undeploy/740799ef-d515-4704-8718-903851c9899e$wjw76aPILglewoYdbq4KK'}},
 'versions': ['0.0.1']}

## 4. Deploying as plug <a id="plug"></a>
Tell the adapter to configure itself as a plug: this generates a number of _assets_ that will be uploaded and define the plug behaviour
* a `plug.json` _manifest_ file th`at defines the name, version, _runtime_, deploy settings, metadata ...
* a `requirements.txt` package dependencies file
* a `main.py` webscript script
* additional scripts we added above, like the `autoencoder.py` and the `model-weights.pt`

#### Initialize adapter for plug deployment (see how to create [adapter](#adapter)).

In [44]:
# choose a local archive location
PLUG_ARCHIVE_LOC = 'autoencoder-pytorch-plug'
# make sure its empty
!rm -fr autoencoder-pytorch

In [45]:
from ml_adapter.torch import V1TorchAdapter
# create an ML adapter to wrap our model
# by using a `weights.pt` postfix we are storing only the weights when serializing the model
MODEL_PATH='model-weights.pt'
adapter = V1TorchAdapter(model=model, model_path='model-weights.pt', location=ARCHIVE_LOC)

# because we store only weights, the adapter archive needs to now about autoencode model class
# Its not recommended to store full serialized models, as these are more brittle with respect versions of python and torch
await adapter.add_script('autoencoder.py')

autoencoder.py <ml_adapter.base.assets.python.PythonScriptAsset>

In [46]:
# the adapter exposes your model with a REST-compatible interface
result = await adapter.call({"instances": [ x_data.tolist() ]}) 
result

{'predictions': [[-0.3229316771030426,
   -0.36001601815223694,
   -0.3960452079772949,
   -0.32631662487983704,
   -0.3968595266342163,
   -0.4243429899215698,
   -0.40644288063049316,
   -0.3829203248023987,
   -0.41355741024017334,
   -0.27493229508399963,
   -0.3954361081123352,
   -0.3798944652080536,
   -0.3340391516685486,
   -0.3709794878959656,
   -0.37912052869796753,
   -0.3987690806388855,
   -0.3101225197315216,
   -0.40983518958091736,
   -0.40134039521217346,
   -0.3952208459377289]]}

In [47]:
## configure any 'memory' or 'cpu' deploy settings 
deploy_overrides = {'limits' : { 'memory': '2G' }, 'requests' : { 'memory' : '1G' }}
## configure the webscript to use our model
adapter = adapter.as_plug({
    'name': MODEL_NAME, 
    'description':'pytorch autoencoder for caats', 
    'deploy' : deploy_overrides                                                                                          
})

In [48]:
await adapter.save()
list(a.path for a in adapter.assets)

['model-weights.pt',
 'webscript.json',
 'requirements.txt',
 'main.py',
 'autoencoder.py',
 'plug.json']

In [49]:
# lets have a look at the generated python plug:
display(Code(filename=f'{ARCHIVE_LOC}/main.py'))

# You could adapt this script to have specific error handling or handling of request/response

# ml_adapter.torch.adapter.V1TorchAdapter model adapter
import os
from ml_adapter.api.data import v1 as V1
from ml_adapter.torch.adapter import V1TorchAdapter

# optional type alias for plug response
StatusAndRawData = tuple[str, V1.V1PredictionResponse|V1.V1ErrorResponse]

STATE_OK = 'PREDICTED'
STATE_NOK = 'FAILED'

MODEL_PATH = os.environ.get('MODEL_PATH', 'model-weights.pt')
MODEL_CLASS = os.environ.get('MODEL_CLASS', 'autoencoder.AutoEncoder')

# Initialize the model adapter.
# Provide a `model` argument if you want to create/load the model yourself.
adapter = V1TorchAdapter(
    model_path=MODEL_PATH, model_class=MODEL_CLASS
)

async def execute(properties: V1.V1Request, console, logger) -> StatusAndRawData:
    try:
        result = await adapter.call(properties)
        return (STATE_OK, result)
    except Exception as err:
        logger.exception(err)
        error_message = str(err)
        console.error(error_message)
        return (STATE_NOK, { 'error': error_message, 'predictions': [] })

In [50]:
list(a.path for a in adapter.assets)

['model-weights.pt',
 'webscript.json',
 'requirements.txt',
 'main.py',
 'autoencoder.py',
 'plug.json']

### Uploading the plug using the SDK
To upload these assets and create a plug, we need to call the [create plug](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Plugs/operation/create_plugs) REST api. 

The code belows uses the `ml_tool` plugin to handle this.
Alternatively you could call `await adapter.save_archive()`
which creates an `autoencoder-pytorch.tar.gz` archive that you can upload as a plug with `curl` or using the waylay console.

In [51]:
from waylay.sdk import WaylayClient

In [52]:
# check the SDK client
client = WaylayClient.from_profile('staging')

In [58]:
try:
    ref = await client.registry.plugs.remove_versions(MODEL_NAME, query={'force': True})
    await asyncio.sleep(5)
except Exception as e:
    print(f'nothing to delete? {e}')

2025-01-21 17:33:52 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/plugs/autoencoderV1?force=true "HTTP/1.1 202 Accepted"


In [59]:
ref = await client.ml_tool.create_plug(adapter)
ref

2025-01-21 17:33:57 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/registry/v2/plugs/?draft=false&comment=&async=true "HTTP/1.1 202 Accepted"


{'message': "Building and deploying plug 'autoencoderV1@0.0.1'",
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$N9ySjGvV_jKsbvAgz7IwY&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/verify/740799ef-d515-4704-8718-903851c9899e$N9ySjGvV_jKsbvAgz7IwY'}},
 'entity': {'createdBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'createdAt': '2025-01-21T16:33:57.825Z',
  'updatedBy': 'users/08e92c94-0a45-4f69-8405-3c2e46dd0cf9',
  'updatedAt': '2025-01-21T16:33:57.839Z',
  'status': 'pending',
  'runtime': {'deprecated': False,
   'upgradable': False,
   'name': 'plug-python3',
   'version': '0.2.0'},
  'deprecated': False,
  'draft': False,
  'plug': {'name': 'autoencoderV1',
   'version': '0.0.1',
   'runtime': 'plug-python3',
   'metadata': {'tags': ['MLAdapter'],
    'documentation': {'description': '',
     'states': [{'name': 'PREDICTED',
       'description': 'T

In [60]:
# wait until the build, deploy and verify jobs for the webscript have finished
# NOTE: this might take quite a few minutes, as the dependencies for torch webscripts are quite big and the images not yet optimised
ref = await client.ml_tool.wait_until_ready(ref)

2025-01-21 17:34:01 INFO     Waiting for autoencoderV1@0.0.1 to be ready:
2025-01-21 17:34:01 INFO     listening on https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$N9ySjGvV_jKsbvAgz7IwY&children=true
2025-01-21 17:34:01 INFO     HTTP Request: GET https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=verify&id=740799ef-d515-4704-8718-903851c9899e$N9ySjGvV_jKsbvAgz7IwY&children=true "HTTP/1.1 200 OK"
2025-01-21 17:34:01 INFO     ack: Listening to events of jobs dependent on job 740799ef-d515-4704-8718-903851c9899e$N9ySjGvV_jKsbvAgz7IwY
2025-01-21 17:34:01 INFO     autoencoderV1@0.0.1 build: active
2025-01-21 17:34:17 INFO     keep-alive: {}
2025-01-21 17:34:47 INFO     keep-alive: {}
2025-01-21 17:35:17 INFO     keep-alive: {}
2025-01-21 17:35:47 INFO     keep-alive: {}
2025-01-21 17:36:17 INFO     keep-alive: {}
2025-01-21 17:36:47 INFO     keep-alive: {}
2025-01-21 17:37:17 INFO     keep-alive: {}
2025-01-21 17:37:47 INFO  

In [61]:
# test the plug invocation
await client.ml_tool.test_plug(ref, x_data.tolist())

2025-01-21 17:39:53 INFO     HTTP Request: POST https://api-aws-dev.waylay.io/rules/v1/sensors/autoencoderV1/versions/0.0.1 "HTTP/1.1 200 OK"


[-0.3229316174983978,
 -0.36001601815223694,
 -0.39604517817497253,
 -0.32631659507751465,
 -0.3968595266342163,
 -0.4243429899215698,
 -0.4064428508281708,
 -0.3829203248023987,
 -0.41355741024017334,
 -0.27493223547935486,
 -0.3954361081123352,
 -0.3798944652080536,
 -0.3340390920639038,
 -0.3709794282913208,
 -0.37912046909332275,
 -0.3987690806388855,
 -0.31012246012687683,
 -0.40983518958091736,
 -0.4013403654098511,
 -0.3952208161354065]

In [62]:
# remove the plug
await client.ml_tool.remove(ref)

2025-01-21 17:39:53 INFO     HTTP Request: DELETE https://api-aws-dev.waylay.io/registry/v2/plugs/autoencoderV1/versions/0.0.1?force=true "HTTP/1.1 202 Accepted"


{'message': "Removing plug version 'autoencoderV1@0.0.1'",
 '_links': {'event': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/events?type=undeploy&id=740799ef-d515-4704-8718-903851c9899e$h3vUl0PHqtrvk8N-2zc0N&children=true'},
  'job': {'href': 'https://api-aws-dev.waylay.io/registry/v2/jobs/undeploy/740799ef-d515-4704-8718-903851c9899e$h3vUl0PHqtrvk8N-2zc0N'}},
 'versions': ['0.0.1']}

#### About `ml_tool`
The `client.ml_tool` methods are essentialy wrappers around the methods of the [registry](https://docs.waylay.io/openapi/public/redocly/registry.html#tag/Webscripts) service. Alternatively you can use the [`client.registry.webscript`](https://github.com/waylayio/waylay-sdk-registry-py) methods directly.

In [63]:
import importlib.metadata
print('versions used:')
for lib in ['waylay-ml-adapter-sdk','waylay-ml-adapter-torch','torch','waylay-sdk-core','waylay-sdk-registry','waylay-sdk-rules']:
    print(f'- {lib}: {importlib.metadata.version(lib)}')

versions used:
- waylay-ml-adapter-sdk: 0.0.9
- waylay-ml-adapter-torch: 0.0.9
- torch: 2.5.1
- waylay-sdk-core: 0.3.2
- waylay-sdk-registry: 2.17.1.20241025
- waylay-sdk-rules: 6.12.0.20241025
